In [1]:
from scipy.sparse import linalg as sp_linalg
from timeit import default_timer as timer
from utilities import *
            
## definitions
# honeycomb and reciprocal lattice vecs
a = 1
u1 = np.array((a * np.sqrt(3), 0))
u2 = np.array((a * np.sqrt(3) / 2, a * 3 / 2))
q1, q2 = get_inv(u1, u2)
v1 = np.array((a * np.sqrt(3) / 2, a / 2))
v2 = np.array((-a * np.sqrt(3) / 2, a / 2))
v3 = np.array((0, a))

# moire lattice vecs
lm1, lm2 = 3, 2
u_moire_1, u_moire_2 = lm1 * u1 + lm2 * u2, (lm1 + lm2) * u2 - lm2 * u1
moire_cell = Cell(u_moire_1, u_moire_2)

# twisting angle
theta_twist = lm1 ** 2 + lm1 * lm2 / 2
theta_twist /= np.sqrt(lm1 ** 2 + lm2 ** 2 + lm1 * lm2)
theta_twist /= lm1
theta_twist = np.arccos(theta_twist)
theta_twist = 2 * theta_twist - np.pi / 3
theta_deg = np.rad2deg(theta_twist)
print(theta_deg)
print(np.rad2deg(np.arccos((lm1 ** 2 + lm2 ** 2 + 4 * lm1 * lm2) / (lm1 ** 2 + lm2 ** 2 + lm1 * lm2) / 2)))

# big parallelogram which contains moire unit cell
l_big_1, l_big_2 = (lm1 + lm2) * 2, (lm1 + 2 * lm2) * 2
u_big_1, u_big_2 = l_big_1 * u1, l_big_2 * u2
big_cell = Cell(u_big_1, u_big_2)

# hoppings
t0 = -2.7
t1 = -2.7 * np.exp(-np.sqrt(3)) * 0.1
t_inter = t0 * 0.0

-13.17355110725886
13.173551107258918


In [3]:
lattice_0, lattice_1 = Lattice(moire_cell), Lattice(moire_cell)
for i1 in range(l_big_1 + 1):
    for i2 in range(l_big_2 + 1):
        vecs = [i1 * u1 + i2 * u2 + big_cell.u_origin, i1 * u1 + i2 * u2 + v3 + big_cell.u_origin]
        for vec in vecs:
            lattice_0.add_site(vec)
        vecs = [rotate_vec(vec, theta_twist) for vec in vecs]
        for vec in vecs:
            lattice_1.add_site(vec)

## NN hoppings
hamdic_00 = Hamdic((lattice_0.num_sites, lattice_0.num_sites), dtype=np.complex128)
vecs = [v1, v2, v3]
for vec in vecs:
    hamdic_00 += lattice_0.get_hamdic_intra(vec, t0)
    hamdic_00 += lattice_0.get_hamdic_intra(-vec, t0)
print("00 shape = {}".format(hamdic_00.shape))
#hamdic_11 = Hamdic((lattice_1.num_sites, lattice_1.num_sites), dtype=np.complex128)
#vecs = [rotate_vec(vec, theta_twist) for vec in vecs]
#for vec in vecs:
#    hamdic_11 += lattice_1.get_hamdic_intra(vec, t0)
#    hamdic_11 += lattice_1.get_hamdic_intra(-vec, t0)
#print("11 shape = {}".format(hamdic_11.shape))
hamdic_11 = None

## NNN hoppings
vecs = [u1, u2-u1, -u2]
for vec in vecs:
    hamdic_00 += lattice_0.get_hamdic_intra(vec, 1j*t1, dtype=np.complex128)
    hamdic_00 += lattice_0.get_hamdic_intra(-vec, -1j*t1, dtype=np.complex128)
#vecs = [rotate_vec(vec, theta_twist) for vec in vecs]
#for vec in vecs:
#    hamdic_11 += lattice_1.get_hamdic_intra(vec, 1j*t1, dtype=np.complex128)
#    hamdic_11 += lattice_1.get_hamdic_intra(-vec, -1j*t1, dtype=np.complex128)

## interlayer hoppings
#hamdic_01 = Hamdic((lattice_0.num_sites, lattice_1.num_sites))
#hamdic_01 += lattice_0.get_hamdic_inter(lattice_1, t_inter, distance_cutoff_sq=a**2)
#print("01 shape = {}".format(hamdic_01.shape))
#hamdic_10 = hamdic_01.get_hermitian_conjugate()
hamdic_01 = None
hamdic_10 = None

hamdic = compose_hamdics([[hamdic_00, hamdic_01], [hamdic_10, hamdic_11]])
for key in hamdic.dic.keys():
    hamdic.dic[key] = hamdic.dic[key].tocsr()
    
## checking hermiticity
q_test = np.array([1.234, 5.678])
ham = lil_matrix(hamdic.shape, dtype=np.complex128)
for key in hamdic.dic.keys():
    u_shift = moire_cell.get_u_shift(key[0], key[1])
    ham += hamdic.dic[key] * np.exp(1j * 2 * np.pi * q_test.dot(u_shift))
non_hermicity = sp_linalg.norm(ham - ham.T.conjugate())
print("|H - H^dag| = {}".format(non_hermicity))

00 shape = (38, 38)
|H - H^dag| = 0.0


In [5]:
## calculating the eigenergies along the 1D line
nq, ns, nd = 51, 10, hamdic.shape[0]
q_arr = [moire_cell.q1 * iq / nq for iq in range(nq)]
q_arr += [moire_cell.q1 + (moire_cell.q2 - moire_cell.q1) * 0.5 * iq / nq for iq in range(nq)]
q_arr += [(moire_cell.q2 + moire_cell.q1) * 0.5 * (1 - iq / nq) for iq in range(nq)]

es_arr = np.zeros((len(q_arr), ns))
for iq in range(len(q_arr)):
    print("iq: {}   ".format(iq), end="\r")
    q_num = q_arr[iq]
    ham = lil_matrix(hamdic.shape, dtype=np.complex128)
    for key in hamdic.dic.keys():
        u_shift = moire_cell.get_u_shift(key[0], key[1])
        ham += hamdic.dic[key] * np.exp(1j * 2 * np.pi * q_num.dot(u_shift))
    ham = ham.tocsc()
    es, vs = sp_linalg.eigsh(ham, k=ns, sigma=0, tol=1e-4)
    es_arr[iq,:] = es[:]

In [8]:
%matplotlib qt5
fig = plt.figure()

fax = fig.add_axes((.15, .15, .75, .75))
for jd in range(ns):
    cax = fax.plot(range(len(q_arr)), es_arr[:,jd], c="red", lw=0, marker='.', ms=2.5)
fax.plot([0, len(q_arr)], [0, 0], c="black", linestyle='-')
fax.scatter([0.5*nq, (1+2/3)*nq, 2.0*nq], [0, 0, 0], s=100, marker='|', c="black")
fax.text((0.52)*nq, 0.0, r"$M$", color="black")
fax.text((1+2/3+0.02)*nq, 0.0, r"$K$", color="black")
fax.text((2.02)*nq, 0.0, r"$M'$", color="black")
    
title = "ens.1d.sparse.lm1={}.lm2={}.theta={:.2f}.t0={:.2f}.t_inter={:.2f}.pdf".format(lm1, lm2, theta_deg, t0, t_inter)
#plt.savefig("figs/"+title)
plt.show()

In [11]:
## plotting the lattice
%matplotlib qt5
fig = plt.figure()

fax = fig.add_axes((.15, .15, .75, .75))
moire_cell.draw(fax)
lattice_0.draw(fax, color_int="red", color_ext="gray", u_offset=np.array([0.01, 0.01]), draw_numbers=False)
lattice_1.draw(fax, color_int="blue", color_ext="gray", u_offset=np.array([-0.01, 0.01]), draw_numbers=False)
big_cell.draw(fax)
big_cell_1 = big_cell.rotate(theta_twist)
big_cell_1.draw(fax)

title = "lattice.lm1={}.lm2={}.pdf".format(lm1, lm2)
plt.savefig("figs/"+title)
plt.show()